In [ ]:
# Foundation
import os
import gc
import json
import random
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Audio
import torchaudio
import librosa
import soundfile as sf

# Transformers
from transformers import AutoTokenizer, ClapModel, AutoFeatureExtractor

# Split
from sklearn.model_selection import train_test_split

# Warnings
warnings.filterwarnings("ignore")

# SEED
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Translated Column Creation with Gemini

In [ ]:
import pandas as pd
import time
import google.generativeai as genai
from tqdm import tqdm
import random

GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-pro')

df = pd.read_csv('vocals_with_emotions.csv')

def translate_to_simpsons_korean(text, max_retries=5):
    prompt = f"""
    Translate this English text to Korean in the style of The Simpsons Korean dubbing:
    "{text}"

    Instructions:
    - Provide ONLY ONE direct Korean translation
    - Do NOT give multiple options or suggestions
    - Do NOT explain your translation choices
    - Do NOT add labels, prefixes or commentary
    - Do NOT include any English text in your response
    - Maintain the humorous and casual tone of The Simpsons
    - Keep the translation natural and conversational

    Your response should contain ONLY the Korean translation text.
    """

    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            translation = response.text.strip()
            return translation
        except Exception as e:
            print(f"Translation error (attempt {attempt+1}/{max_retries}): {e}")

df['korean_text'] = None

start_idx = 0
for i in range(len(df), 0, -10):
    if i % 10 != 0:
        continue
    if pd.io.common.file_exists(f'vocals_with_emotions_korean_partial_{i}.csv'):
        print(f"Found partial file at index {i}, resuming from there.")
        partial_df = pd.read_csv(f'vocals_with_emotions_korean_partial_{i}.csv')
        df.loc[:i-1, 'korean_text'] = partial_df.loc[:i-1, 'korean_text']
        start_idx = i
        break

for idx, row in tqdm(df.iloc[start_idx:].iterrows(), total=len(df)-start_idx, desc="Translating"):
    english_text = row['text']

    if pd.isna(english_text) or english_text.strip() == '':
        df.at[idx, 'korean_text'] = ''
        continue

    korean_text = translate_to_simpsons_korean(english_text)
    df.at[idx, 'korean_text'] = korean_text

    print(f"\n{idx+1}/{len(df)} completed")
    print(f"Original: '{english_text}'")
    print(f"Translation: '{korean_text}'")

df.to_csv('vocals_with_emotions_korean_add_shorts.csv', index=False)

print("\nTranslation completed! Results saved to 'vocals_with_emotions_korean.csv'")

sample_df = df[['text', 'korean_text']].sample(min(5, len(df)))
for _, row in sample_df.iterrows():
    print(f"Original: '{row['text']}'")
    print(f"Translation: '{row['korean_text']}'")
    print('-' * 50)

Translating:   1%|          | 1/118 [00:01<02:53,  1.48s/it]


1/118 completed
Original: 'What kind of lunch is this?'
Translation: '이게 무슨 점심이야?'


Translating:   2%|▏         | 2/118 [00:02<02:25,  1.25s/it]


2/118 completed
Original: 'A drawing of a sandwich.'
Translation: '샌드위치 그림이네.'


Translating:   3%|▎         | 3/118 [00:03<02:18,  1.20s/it]

...


185/118 completed
Original: 'I know you're upset right now, so I'll pretend you didn't say that.'
Translation: '지금 화났으니까, 못 들은 걸로 할게.'


Translating:  99%|█████████▉| 117/118 [02:13<00:01,  1.19s/it]


186/118 completed
Original: 'You little...'
Translation: '이 쪼끄만…'


Translating: 100%|██████████| 118/118 [02:14<00:00,  1.14s/it]


188/118 completed
Original: 'You little...'
Translation: '이 쪼끄만…'

Translation completed! Results saved to 'vocals_with_emotions_korean.csv'
Original: 'Use the dino wand.'
Translation: '공룡 막대기 써.'
--------------------------------------------------
Original: 'Ha ha, Jesus!'
Translation: '아이고, 세상에!'
--------------------------------------------------
Original: 'Today is the anniversary of the dark Stanley murders.'
Translation: '으악! 오늘이 바로 그 끔찍한 스탠리 가 살인사건 기념일이라니!'
--------------------------------------------------
Original: ' Get off!'
Translation: '내려!'
--------------------------------------------------
Original: 'A frightened Obeisius and his new work friends were taken to Rome.'
Translation: '겁먹은 오베이셔스랑 걔 새 직장 동료들이 로마로 끌려갔다네!'
--------------------------------------------------
